## 2.1.2. Bài làm mẫu  
### Bài toán 1: Xây dựng cây quyết định và rừng cây với dữ liệu "Default of Credit Card Clients"
---
**Nguồn dữ liệu:** [https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset](https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset)

#### Nhiệm vụ 1: Xây dựng cây quyết định bằng thư viện Scikit-Learn

### 1. Cài đặt và import các thư viện cần thiết

In [23]:
!pip install numpy pandas matplotlib scikit-learn graphviz


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import graphviz
from sklearn import tree
from sklearn.model_selection import train_test_split

%matplotlib inline
mpl.rcParams['figure.dpi'] = 100

### 2. Nạp dữ liệu và loại bỏ các biến không cần thiết

In [25]:
import pandas as pd
df = pd.read_csv('UCI_Credit_Card.csv')
df.head()
features_response = df.columns.tolist()

items_to_remove = [
    'ID', 'SEX', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
    'EDUCATION_CAT', 'graduate school', 'high school', 'none',
    'others', 'university'
]

features_response = [item for item in features_response if item not in items_to_remove]
features_response

['LIMIT_BAL',
 'EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_0',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6',
 'default.payment.next.month']

### 3. Chuẩn bị dữ liệu cho tập train và tập test

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import tree

X_train, X_test, y_train, y_test = train_test_split(
    df[features_response[:-1]].values,
    df['default.payment.next.month'].values,
    test_size=0.2,
    random_state=24
)

NameError: name 'df' is not defined

### 4. Xây dựng cây quyết định từ lớp DecisionTreeClassifier có trong thư viện Scikit-Learn

In [ ]:
dt = tree.DecisionTreeClassifier(max_depth=2)
dt.fit(X_train, y_train)

In [ ]:
import graphviz
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
tree.plot_tree(
    dt,
    feature_names=features_response[:-1],
    class_names=['Not defaulted', 'Defaulted'],
    filled=True
)
plt.show()

In [ ]:
plt.savefig("decision_tree.png", dpi=300)

### Nhiệm vụ 2: Tìm tham số tối ưu cho cây quyết định bằng GridSearchCV và vẽ biểu đồ đánh giá mô 


### 1. Chuẩn bị dữ liệu

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import graphviz
from sklearn import tree
from sklearn.model_selection import train_test_split

%matplotlib inline
mpl.rcParams['figure.dpi'] = 100

In [ ]:
import pandas as pd
df = pd.read_csv('UCI_Credit_Card.csv')
df.head()
features_response = df.columns.tolist()

items_to_remove = [
    'ID', 'SEX', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
    'EDUCATION_CAT', 'graduate school', 'high school', 'none',
    'others', 'university'
]

features_response = [item for item in features_response if item not in items_to_remove]
features_response

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import tree

X_train, X_test, y_train, y_test = train_test_split(
    df[features_response[:-1]].values,
    df['default.payment.next.month'].values,
    test_size=0.2,
    random_state=24
)

### 2. Tạo cây quyết định và xác định các giá trị tham số có thể chọn để tìm tham số tối ưu

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'max_depth': [1, 2, 4, 6, 8, 10, 12]}
dt = tree.DecisionTreeClassifier(random_state=24)

cv = GridSearchCV(
    dt,
    param_grid=params,
    scoring='roc_auc',       
    n_jobs=None,
    refit=True,
    cv=4,                     
    verbose=1,
    error_score=np.nan,
    return_train_score=True
)

cv.fit(X_train, y_train)

## Giải thích tham số:

scoring='roc_auc': dùng diện tích dưới đường cong ROC để đánh giá

cv=4: chia dữ liệu thành 4 phần để cross-validation

return_train_score=True: lưu cả điểm train để so sánh bias và variance

### 3. Vẽ biểu đồ đánh giá mô hình với các tham số chiều sâu của cây khác nhau

In [ ]:
cv_results_df = pd.DataFrame(cv.cv_results_)
cv_results_df[['param_max_depth', 'mean_train_score', 'mean_test_score']].head()

ax = plt.axes()

# Vẽ đường train
ax.errorbar(
    cv_results_df['param_max_depth'],
    cv_results_df['mean_train_score'],
    yerr=cv_results_df['std_train_score']/np.sqrt(4),
    label='Mean ± 1 SE training scores'
)

# Vẽ đường test
ax.errorbar(
    cv_results_df['param_max_depth'],
    cv_results_df['mean_test_score'],
    yerr=cv_results_df['std_test_score']/np.sqrt(4),
    label='Mean ± 1 SE testing scores'
)

ax.legend()
plt.xlabel('max_depth')
plt.ylabel('ROC AUC')
plt.title('Đánh giá hiệu quả Decision Tree với các độ sâu khác nhau')
plt.savefig("decision_tree_depth.png", dpi=300, bbox_inches='tight')
plt.show()